In [1]:
import numpy as np
import pandas as pd

from arrow import Arrow
from bcpp_community import communities
from bcpp_status.models import StatusHistory
from bcpp_subject.models import SubjectVisit, SubjectConsent
from datetime import datetime
from django.db import connection
from edc_constants.constants import YES, NO, NEG, UNK
from edc_pdutils.model_to_dataframe import ModelToDataframe
from pprint import pprint

from member.models import AbsentMember

In [16]:
class SubjectModelToDataframe(ModelToDataframe):
    
    columns = {k: k for k in ['subject_identifier', 'gender']}
    
    def __init__(self, columns=None, **kwargs):
        if columns:
            self.columns = {k: k for k in columns}
        super().__init__(**kwargs)

In [3]:
df_absent = ModelToDataframe(model='member.absentmember', drop_sys_columns=True).dataframe

In [5]:
df_absent.groupby('reason').size()

reason
OTHER                                                         1704
goes to school in village and comes home daily                 199
goes to school outside village and comes home daily             88
goes to school outside village and comes home irregularly      303
goes to school outside village and comes home monthly          112
goes to school outside village and comes home on weekends       22
gone visiting (relatives,holidays,weddings,funerals)          2050
stays at lands or cattlepost                                   660
stepped out(shops, errands etc)                                663
works in village and comes home daily                          351
works outside village and comes home daily                     505
works outside village and comes home irregularly               612
works outside village and comes home monthly                   211
works outside village and comes home on weekends                89
dtype: int64

In [7]:
df_absent[df_absent['reason'] == 'OTHER'].groupby('reason_other').size()

reason_other
                                       20
ABSENT WITHOUT NOTICE                   4
ADMITED AT BOKAMOSO HOSPITAL            1
APPOINTED LATER DATE                    1
AT CHURCH CAMP                          1
AT MAURITIUS                            3
AT WORK                                 3
ATTENDED A FUNERAL                      1
ATTENDED FAMILY MATTERS IN KANYE        1
AVAILABLE ON SUNDAYS ONLY               2
AVAILABLE ON WEEKENDS                   2
BDF TRAINING AT FRANCISTOWN             2
BOARDING SCHOOL OUTSIDE THE VILLAGE     3
BUSY                                    3
BUSY AT THE FIELD                       1
BUSY IN THE FUNERAL,                    2
BUSY NOW                                1
BUSY TODAY                             10
CAN NOT  BE FOUND                       2
CATTLE POST                             1
CHECK UP AT HOME                        1
CONTACTS NOT AVAILABLE                 18
DOES NOT ANSWER THE PHONE,AFTER APP     1
DONT KNOW AS THERES N

In [8]:
# Load the table
df_refused = ModelToDataframe(model='member.refusedmember', drop_sys_columns=True).dataframe

In [9]:
# Group by reason
df_refused.groupby('reason').size()

reason
Already on HAART                                    9
DWTA                                                5
Fear of needles                                     5
I already know I am HIV-positive                   20
I already know my status                           59
I am afraid my information will not be private      5
I am afraid of testing                              7
I am not ready to test                             25
I don't have time                                  35
I don't want to answer the questions                9
I don't want to have the blood drawn               11
I don't want to take part                         129
I haven't had a chance to think about it            2
I want to test where i always test                 15
Illiterate does not want a witness                  1
OTHER                                              33
not_sure                                            1
dtype: int64

In [10]:
# Group by reason_other for those where reason = OTHER
df_refused[df_refused['reason'] == 'OTHER'].groupby('reason_other').size()

reason_other
ALREADY TAKE PART WITH HAVARD ESS      1
ALWAYS SAYING SHE IS AT THE FIELD      1
CANT DO IT WITHOUT HUSBANDS APPROVL    1
CHASED ME FROM HIS HOUSEHOLD           1
DO HAVE INTEREST                       1
DOES NOT WANT TO TEST                  1
I AM NOT INTERESTED                    1
I DON'T WANT TO PARTICIPATE            1
I DON'T WANT TO TAKE PART              1
I HAVE MY OWN DOCTOR                   1
I HAVE NO INTEREST                     1
JUST DONT WANT TO TEST AT ALL          1
NO LONGER INTERESTED                   1
NOT INTERESTED                         6
PARTICIPANT SAYS NOT TO BE BOTHERED    1
PERSONAL REASONS                       1
REFUSING TO GET OUT OF THE HOUSE       1
REFUSING TO TALK TO US                 1
RELIGIOUS BELIEVES                     3
RELOCATED                              1
SHE IS A SANGOMA,NO PIERCINGS          1
TRAUMA DUE TO CAR RECENT ACCIDENT      1
WILL DO SOME OTHER TIME SOMEWHERE      1
WILL TEST AT TEBELOPELE VCTC           1
i d

In [18]:
# absent_reason is not none
cols = ['subject_identifier', 'internal_identifier', 'absent', 'refused', 'undecided']
df_member = SubjectModelToDataframe(model='member.householdmember', columns=cols).dataframe

In [21]:
df_member.groupby(['absent', 'refused', 'undecided']).size()

absent  refused  undecided
False   False    False        72855
                 True           342
        True     False         1636
                 True            21
True    False    False         8830
                 True            96
        True     False          130
                 True             3
dtype: int64